In [10]:

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import robotexclusionrulesparser
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


def is_allowed(url, user_agent='*'):
    parsed_url = requests.utils.urlparse(url)
    base_url = f"{parsed_url.scheme}://{parsed_url.netloc}/robots.txt"

    rp = robotexclusionrulesparser.RobotExclusionRulesParser()
    try:
        robots_txt = requests.get(base_url, timeout=5).text
        rp.parse(robots_txt)
        return rp.is_allowed(user_agent, url)
    except Exception as e:
        print(f"Could not fetch robots.txt for {url}: {e}")
        return True


def fetch_dynamic_content(url):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    driver = webdriver.Chrome('chromedriver', options=options)

    try:
        driver.get(url)
        html = driver.page_source
    finally:
        driver.quit()
    return html


def crawler_with_delay(start_url, delay=2, max_pages=5, use_selenium=False):
    visited = set()
    to_visit = [start_url]
    index = {}

    while to_visit and len(visited) < max_pages:
        url = to_visit.pop(0)
        if url not in visited:
            if is_allowed(url):
                try:
                    if use_selenium:
                        html = fetch_dynamic_content(url)
                        soup = BeautifulSoup(html, 'html.parser')
                    else:
                        response = requests.get(url, timeout=10)
                        if response.status_code == 200:
                            soup = BeautifulSoup(response.text, 'html.parser')
                        else:
                            print(f"Failed to fetch {url} with status code {response.status_code}")
                            continue

                    title = soup.title.string if soup.title else 'No Title'
                    index[url] = title
                    print(f"Crawled: {url} -> {title}")

                    for link in soup.find_all('a', href=True):
                        absolute_link = urljoin(url, link['href'])
                        if absolute_link not in visited and absolute_link not in to_visit:
                            to_visit.append(absolute_link)

                except Exception as e:
                    print(f"Failed to crawl {url}: {e}")

                visited.add(url)
                time.sleep(delay)
            else:
                print(f"Crawling disallowed for: {url}")
    return index


if __name__ == "__main__":
    start_url = "https://www.youtube.com/"
    print("Allowed to crawl:", is_allowed(start_url))
    indexed_pages = crawler_with_delay(start_url, delay=2, max_pages=3, use_selenium=False)

    print("\nIndexed Pages:")
    for url, title in indexed_pages.items():
        print(url, ":", title)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 115.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 38.6 MB/s eta 0:00:00
  Created wheel for robotexclusionrulesparser: filename=robotexclusionrulesparser-1.7.1-py3-none-any.whl size=12057 sha256=ddc7054362d9b01026d63b8f2a2e16e9791c59fb562d878fe5ac7fec6d676493
  Stored in directory: /root/.cache/pip/wheels/51/db/cd/5be41448be8f9f645f7467876ed03d2d85434bb3061fa4ca2d
Successfully built robotexclusionrulesparser
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u